# Synthetic ice sheet

In this demo we'll run an idealized experiment inspired by the paper [*Fjord insertion into continental margins
driven by topographic steering of ice*](https://www.nature.com/articles/ngeo201) by Kessler et al. (2008).
Their work simulated the evolution of an entire ice sheet on millenial timescales, with the added twist that the bedrock topography freely evolved under the influence of erosion. We will simulate an ice sheet on a similar bedrock topography, but the bed will stay constant and we will not include erosion.

Starting with a rough guess for the initial ice thickness, we'll solve the *diagnostic equation* for the velocity throughout the ice sheet.
We'll then solve the *prognostic equation* to obtain the ice thickness at a slightly later time.
By assuming a given rate of ice accumulation and melting, we can successively solve the prognostic and diagnostic equations until the system is close to a steady state.

The whole simulation can be divided into three parts:

* Define the shape of the ice sheet and get a triangulation of the interior.
* Define the initial guess for the ice thickness and velocity, and set a value of the rheology and accumulation rate.
* Set the total simulation time and the number of time steps, and then iteratively update the ice thickness and velocity at each time step.

This is a pretty common workflow for a predictive model.

In [ ]:
import numpy as np
import firedrake
from firedrake import (
    Constant, Function, sqrt, exp, max_value, inner, grad, jump, dx, ds, dS
)
import irksome
from irksome import Dt
import icepack.plot
from tqdm.notebook import tqdm, trange
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

mesh = firedrake.UnitDiskMesh(5)
R = 250e3
mesh.coordinates.dat.data[:] *= R

In [ ]:
degree = 0

if degree == 0:
    element = firedrake.FiniteElement("DG", "triangle", 0)
else:
    bernstein = firedrake.FiniteElement("Bernstein", "triangle", degree)
    element = firedrake.BrokenElement(bernstein)
Q = firedrake.FunctionSpace(mesh, element)

cg = firedrake.FiniteElement("CG", "triangle", degree + 1)
S = firedrake.FunctionSpace(mesh, cg)

In [ ]:
x, y = firedrake.SpatialCoordinate(mesh)
r = sqrt(x**2 + y**2)

# Plateau elevation
b_base = Constant(400)

# Max elevation
b_max = Constant(1400)

# Radius of the plateau interior
ro = Constant(125e3)

# Radius of the ridge
Ro = Constant(200e3)

def tanh(z):
    return (exp(z) - exp(-z)) / (exp(z) + exp(-z))

def θ(z):
    return (tanh(z) + 1) / 2

def sech(z):
    return 2 / (exp(z) + exp(-z))

In [ ]:
a = Constant(50e3)
ξ = (sqrt(x**2 + y**2) - ro) / a

b_expr_plateau = b_base * (1 - θ(3 * ξ))
b = Function(S).interpolate(b_expr_plateau)

fig, axes = icepack.plot.subplots()
kw = {"vmin": -600, "vmax": +1200}
colors = firedrake.tripcolor(b, axes=axes, **kw)
fig.colorbar(colors, label="meters above sea level")
axes.set_title("Bed Plateau");

In [ ]:
ζ = (r - Ro) / Ro

b_expr_ridge = (b_max - b_base) * sech(3 * ξ)
b_expr = b_expr_plateau + b_expr_ridge
b = Function(S).interpolate(b_expr)

fig, axes = icepack.plot.subplots()
contours = firedrake.tripcolor(b, axes=axes, **kw)
fig.colorbar(contours, label="meters above sea level")
axes.set_title("Bed Plateau and Ridge");

In [ ]:
ρ1 = Constant(1 / 4)
μ1 = 1 - ρ1 * θ(3 * (x - ro / 4) / a) * sech(2 * y / a)

ρ2 = Constant(3 / 8)
μ2 = 1 - ρ2 * θ(3 * (y - ro / 4) / a) * sech(2 * x / a)

ρ3 = Constant(1 / 2)
μ3 = 1 - ρ3 * θ(3 * (-x + ro / 4) / a) * sech(2 * y / a)

ρ4 = Constant(5 / 8)
μ4 = 1 - ρ4 * θ(3 * (-y + ro / 4) / a) * sech(2 * x / a)

μ = μ1 * μ2 * μ3 * μ4

δb = Constant(480) / (1 - Ro / R)

b_expr_valleys = (b_max - b_base) * sech(3 * ξ) * μ - θ(5 * ζ) * δb * ζ
b_expr = b_expr_plateau + b_expr_valleys
b = Function(S).interpolate(b_expr)

fig, axes = icepack.plot.subplots()
contours = firedrake.tripcolor(b, axes=axes, **kw)
fig.colorbar(contours, label="meters above sea level")
axes.set_title("Bed Plateau, \n Ridge, and Valleys");

In [ ]:
# Surface elevation
max_radius = Constant(195e3)
dome_height = Constant(2.4e3)
dome = dome_height * max_value(1 - (x**2 + y**2) / max_radius**2, 0)
s0 = Function(Q).interpolate(dome)

# Thickness
h_expr = max_value(s0 - b, 0) * firedrake.conditional(b >= 0, 1, 0)
h0 = Function(Q).interpolate(h_expr)

In [ ]:
fig, axes = icepack.plot.subplots()
colors = firedrake.tripcolor(s0, axes=axes)
fig.colorbar(colors, label="meters above sea level")
axes.set_title("Initial Ice \n Surface Elevation");

In [ ]:
fig, axes = icepack.plot.subplots()
colors = firedrake.tripcolor(h0, axes=axes, cmap="Blues")
fig.colorbar(colors, label="meters")
axes.set_title("Initial Ice Thickness Contours \n overlain on Bed Surface");

### Modeling

In [ ]:
T = Constant(273.15 - 5)
A = icepack.rate_factor(T)

In [ ]:
from icepack.constants import glen_flow_law, ice_density, gravity

ρ_I = Constant(ice_density)
g = Constant(gravity)
n = Constant(glen_flow_law)

h = h0.copy(deepcopy=True)
P = ρ_I * g * h
s = b + h
u = -2 * h * A / (n + 2) * P**n * inner(grad(s), grad(s)) ** ((n - 1) / 2) * grad(s)

In [ ]:
a = Constant(0.0)
m = Constant(0.0)

q = firedrake.TestFunction(Q)
F_cells = (Dt(h) * q - inner(h * u, grad(q)) - (a - m) * q) * dx

ν = firedrake.FacetNormal(mesh)
f = h * max_value(0, inner(u, ν)) * ν
F_facets = jump(f * q, ν) * dS

F = F_cells + F_facets

In [ ]:
tableau = irksome.BackwardEuler()
t = Constant(0.0)
dt = Constant(1.0)

lower = firedrake.Function(Q)
upper = firedrake.Function(Q)
lower.assign(0.0)
upper.assign(+np.inf)
bounds = ("stage", lower, upper)

bparams = {
    "solver_parameters": {
        "snes_monitor": ":ice-sheet.log",
        "snes_type": "vinewtonrsls",
        "snes_max_it": 200,
        "ksp_type": "gmres",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "mumps",
    },
    "form_compiler_parameters": {"quadrature_degree": 6},
    "stage_type": "value",
    "basis_type": "Bernstein",
    "bounds": bounds,
}

solver = irksome.TimeStepper(F, tableau, t, dt, h, **bparams)

In [ ]:
hs = [h.copy(deepcopy=True)]
final_time = 1000.0
num_steps = int(final_time / float(dt))
for step in trange(num_steps):
    solver.advance()
    hs.append(h.copy(deepcopy=True))

In [ ]:
%%capture

fig, axes = icepack.plot.subplots()
kw = {"num_sample_points": 4, "shading": "gouraud", "cmap": "Blues"}
colors = firedrake.tripcolor(hs[0], **kw, axes=axes)
fn_plotter = firedrake.FunctionPlotter(mesh, num_sample_points=4)
animate = lambda h: colors.set_array(fn_plotter(h))

In [ ]:
animation = FuncAnimation(fig, animate, tqdm(hs), interval=1e3/60)

In [ ]:
HTML(animation.to_html5_video())